# Import Modules

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
from PIL import Image

# Load Train and Val Dataset

In [2]:
pri_train = sorted(os.listdir('./Dataset/MTCNNCroppedFaces/train/private/'))
pub_train = sorted(os.listdir('./Dataset/MTCNNCroppedFaces/train/public/'))
semi_train = sorted(os.listdir('./Dataset/MTCNNCroppedFaces/train/semiprivate/'))

pri_train_filename = [x.split('.')[0] for x in pri_train]
pub_train_filename = [x.split('.')[0] for x in pub_train]
semi_train_filename = [x.split('.')[0] for x in semi_train]


print(len(pri_train))
print(len(pub_train))
print(len(semi_train))
print(pri_train_filename[:10])
print(pub_train_filename[:10])
print(semi_train_filename[:10])

521
464
657
['00000ae5e7fcc87222f1fb6f221e7501558e5b08', '00ae730b3a1b68358d4bda73978b14707cd1ef6c', '00af5da9ae2bca79dd46100a485fd13bba848972', '00b1297d36c7dd6470d9439df065a10b3b241f40', '00c2a1e3df8c9ed0e6b17447f43d9ede97aa6b6b', '00c3b8f6898f99bfa77bf10a1492fb959336c158', '00d40a80e5cbcc09883ce1ad79e79b16879786e5', '026ffb1b976d6ab867aabf480a9b0dcf287a72fa', '05apr09a', '063dba6d9617b6265e52f62ca8fe99bd7efaf6b3']
['00bc0b7df6951edbe08ac69cfb00f0b7a9927a31', '00bde3f3037497aaa1e665ba1b23fc149ec9081f', '00c5d8c5b83e61cfcf1bd0dffb0911230d25ce7b', '00e0cca23e2ff5b4f8310b7ba612646760eabf0f', '00fdfe0a19143c97cabeb245b2ca8b7525930885', '01134c2e6401d8cead6f0192cf36bbcf15e91755', '018415d0b71b8f2d85c7f685ff5ebbc8489a9eb6', '05e5fc6e6a70ff619847cc18a531e9fcae180db6', '074f843a6ea76ec60d89181b96f2fe77aa652831', '07ff674977f00fb1b2718d9dcdabe3663dff668a']
['00ce1cf572bcf97c138129087291ba7cce7ed8b3', '0339520eef09650d55879247cba49c752a2c0b1e', '06ad54135a0143aedc11e3c6d13defc1e69097d2', '06af

In [3]:
pri_val = sorted(os.listdir('./Dataset/MTCNNCroppedFaces/val/private/'))
pub_val = sorted(os.listdir('./Dataset/MTCNNCroppedFaces/val/public/'))
semi_val = sorted(os.listdir('./Dataset/MTCNNCroppedFaces/val/semiprivate/'))

pri_val_filename = [x.split('.')[0] for x in pri_val]
pub_val_filename = [x.split('.')[0] for x in pub_val]
semi_val_filename = [x.split('.')[0] for x in semi_val]


print(len(pri_val))
print(len(pub_val))
print(len(semi_val))
print(pri_val_filename[:10])
print(pub_val_filename[:10])
print(semi_val_filename[:10])


#dataset_path = './MTCNNCroppedFaces/'
#pathname = sorted(os.listdir(dataset_path))
#filename = [x.split('.')[0] for x in pathname]
#print(filename[:10])

70
96
122
['04085231fc015457761e587ee53e4d1be6eb3666', '11708847396_9383898125_b', '12772838113_10d6e384c0_b', '12772897004_3573abfe77_b', '137300', '13cfa59c9fbc82b5cc6866c4d525f35bd1003777', '143eb889d808e38195c2ac1aa20139e285905aa0', '14938114374_89dd0c0566_b', '15061614238_70f0da58c2_b', '15247938872_998df4b740_b']
['052e744bb87e87adebaf05d4da1fea43c91519be', '060cbff33fb752f1e66ab4154e36b18e98261454', '064af8c6ddf394493b57f6bcab1b0b5c8b30945e', '091bb234cd3396f4324578f0247f21bb5ca5155e', '11b97c7d54de997275c1a81c6c858987db287aa0', '11bcfa23e6a029e15b5f1a18137cabdc2cca6a93', '140408_gma_prom1_16x9_992', '16_69_18_Portrait_of_attractive_young_Student_Girl_in_College_Hallway', '18076403_G', '20160604-TAICOCLUB-013']
['059a1dc3bea8729ee6c43ba3b8280463cf052a6a', '10173736_10152323620229841_233701820_n', '101798768-india_students_2', '1128558_orig', '1280-508213235-teacher-talking-with-students-in-chemistry-lab', '130135_iStock-106457063', '138284_GettyImages-108878398', '180726_5577_cy

# Align Faces of Training Dataset

In [4]:
for j in range(len(pri_train)):
    print(j)
    data = np.load("./Dataset/MTCNNCroppedFaces/train/private/"+str(pri_train_filename[j])+".npz")
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"./Dataset/MTCNNAlignedCroppedFaces/train/private/"+str(pri_train_filename[j])+"_"+str(i)+".jpg")
    j = j+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
for j in range(len(pub_train)):
    
    print(j)
    data = np.load("./Dataset/MTCNNCroppedFaces/train/public/"+str(pub_train_filename[j])+".npz")
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"./Dataset/MTCNNAlignedCroppedFaces/train/public/"+str(pub_train_filename[j])+"_"+str(i)+".jpg")
    j = j+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
for j in range(len(semi_train)):
    print(j)
    data = np.load("./Dataset/MTCNNCroppedFaces/train/semiprivate/"+str(semi_train_filename[j])+".npz")
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"./Dataset/MTCNNAlignedCroppedFaces/train/semiprivate/"+str(semi_train_filename[j])+"_"+str(i)+".jpg")
    j = j+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Align Faces of Val Dataset

In [7]:
for j in range(len(pri_val)):
    print(j)
    data = np.load("./Dataset/MTCNNCroppedFaces/val/private/"+str(pri_val_filename[j])+".npz")
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"./Dataset/MTCNNAlignedCroppedFaces/val/private/"+str(pri_val_filename[j])+"_"+str(i)+".jpg")
    j = j+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


In [8]:
for j in range(len(pub_val)):
    print(j)
    data = np.load("./Dataset/MTCNNCroppedFaces/val/public/"+str(pub_val_filename[j])+".npz")
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"./Dataset/MTCNNAlignedCroppedFaces/val/public/"+str(pub_val_filename[j])+"_"+str(i)+".jpg")
    j = j+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95


In [9]:
for j in range(len(semi_val)):
    print(j)
    data = np.load("./Dataset/MTCNNCroppedFaces/val/semiprivate/"+str(semi_val_filename[j])+".npz")
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"./Dataset/MTCNNAlignedCroppedFaces/val/semiprivate/"+str(semi_val_filename[j])+"_"+str(i)+".jpg")
    j = j+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
